# 1. Install Dependencies and Setup

In [ ]:
# !pip install tensorflow==2.10.0 tensorflow-gpu==2.10.0 opencv-python matplotlib

In [ ]:
import os
import cv2
import tensorflow as tf
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Avoid OOM errors by settings GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

# 2. Load Data

### 2.1 Load Images

In [ ]:
def load_images(x):
  byte_img = tf.io.read_file(x)
  img = tf.io.decode_jpeg(byte_img)
  return img

In [ ]:
IMAGE_TRAIN_PATH = os.path.join('aug_data', 'train', 'images')
train_images = tf.data.Dataset.list_files(os.path.join(IMAGE_TRAIN_PATH, '*.jpg'), shuffle=False)
train_images.as_numpy_iterator().next()

In [ ]:
train_images = train_images.map(load_images)
plt.imshow(train_images.as_numpy_iterator().next())

In [ ]:
train_images.as_numpy_iterator().next().shape

In [ ]:
train_images = tf.data.Dataset.list_files(os.path.join('aug_data', 'train', 'images', '*.jpg'), shuffle=False)
train_images = train_images.map(load_images)
train_images = train_images.map(lambda x: tf.image.resize(x, (250, 250)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files(os.path.join('aug_data', 'test', 'images', '*.jpg'), shuffle=False)
test_images = test_images.map(load_images)
test_images = test_images.map(lambda x: tf.image.resize(x, (250, 250)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files(os.path.join(os.path.join('aug_data', 'val', 'images', '*.jpg')), shuffle=False)
val_images = val_images.map(load_images)
val_images = val_images.map(lambda x: tf.image.resize(x, (250, 250)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,20))

ax[0].imshow(train_images.as_numpy_iterator().next())
ax[0].set_title('Train')

ax[1].imshow(test_images.as_numpy_iterator().next())
ax[1].set_title('Test')

ax[2].imshow(val_images.as_numpy_iterator().next())
ax[2].set_title('Val')

### 2.2 Prepare labels

In [ ]:
def load_labels(label_path):
  with open(label_path.numpy(), 'r', encoding='utf-8') as f:
    label = json.load(f)
    
  return [label['keypoints']]

In [ ]:
train_labels = tf.data.Dataset.list_files(os.path.join('aug_data', 'train', 'labels', '*.json'), shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files(os.path.join('aug_data', 'test', 'labels', '*.json'), shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files(os.path.join('aug_data', 'val', 'labels', '*.json'), shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
val_labels.as_numpy_iterator().next()

### 2.3 Combine Labels and Images

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(8000)
train = train.batch(16)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(2000)
test = test.batch(16)
test = train.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(2200)
val = val.batch(16)
val = val.prefetch(4)

### 2.4 View Sample

In [ ]:
data_sample = train.as_numpy_iterator()

In [ ]:
res = data_sample.next()

In [ ]:
res

In [ ]:
np.multiply(res[1][0][1], [250, 250, 250, 250]).astype(int)

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
  sample_image = res[0][idx]
  sample_coords = res[1][0][idx]

  cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 5, (255, 0, 0,), -1)
  cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 5, (0, 255, 0,), -1)

  ax[idx].imshow(sample_image)

# 3. Build Model

### 3.1 Create Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0


In [ ]:
input_layer = Input(shape=(250, 250, 3))
input_layer

In [ ]:
backbone = EfficientNetV2B0(include_top=False)(input_layer)
backbone

In [ ]:
# Hidden Layers Here
cnn1 = Conv2D(512, 3, padding='same', activation='relu')(backbone)
cnn1.shape

In [ ]:
cnn2 = Conv2D(512, 3, padding='same', activation='relu')(cnn1)
cnn2.shape

In [ ]:
cnn3 = Conv2D(256, 3, 2, padding='same', activation='relu')(cnn2)
cnn3.shape

In [ ]:
cnn4 = Conv2D(256, 2, 2, activation='relu')(cnn3)
cnn4.shape

In [ ]:
cnn5 = Conv2D(4, 2, 2)(cnn4)
cnn5.shape

In [ ]:
reshape = Reshape((4,))(cnn5)
reshape.shape

In [ ]:
model = Sequential([
  Input(shape=(250,250,3)),
  EfficientNetV2B0(include_top=False, input_shape=(250,250, 3)),
  Conv2D(16, 3, padding='same', activation='relu'),
  Conv2D(16, 3, padding='same', activation='relu'),
  Conv2D(8, 3, 2, padding='same', activation='relu'),
  Conv2D(8, 2, 2, activation='relu'),
  Dropout(0.05),
  Conv2D(4, 2, 2),
  Reshape((4,))
])

In [ ]:
model.summary()

### 3.2 Setup Losses and Optimizer

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.0007)
loss = tf.keras.losses.MeanSquaredError()

In [ ]:
model.compile(optimizer, loss)

### 3.3 Sense Check Predicitons

In [ ]:
x, y  = train.as_numpy_iterator().next()

In [ ]:
x.shape

In [ ]:
coordinates = model.predict(x)
coordinates

### 3.4 Train the Model

In [ ]:
# train for 100 epochs
hist = model.fit(train, epochs=1, validation_data=val, batch_size=16)

# 4. Review Performace

In [ ]:
hist.history

In [ ]:
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val loss')
plt.suptitle('Loss Plot - Training')
plt.legend()
plt.show()

### 4.2 Make Predictions on Test

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

In [ ]:
yhat = model.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
  sample_image = test_sample[0][idx]
  sample_coords = yhat[idx]

  cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 5, (255, 0, 0,), -1)
  cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 5, (0, 255, 0,), -1)

  ax[idx].imshow(sample_image)

### 4.3 Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('eyetracker5epochs.h5')

In [ ]:
model = load_model('eyetracker5epochs.h5')

# 5. Real Time

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
  _, frame = cap.read()

  frame = frame[50:500, 50:500,:]
  rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  resized = cv2.resize(rgb_img, (250, 250))

  yhat = model.predict(np.expand_dims(resized/255.0))
  sample_coords = yhat[0, :4]

  cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)), 5, (255, 0, 0,), -1)
  cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 5, (0, 255, 0,), -1)

  cv2.imshow('EyeTrack', frame)

  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()